In [8]:
pip install psycopg2

     ---------------------------------------- 1.2/1.2 MB 12.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import psycopg2
import pandas as pd

***Access Historical Data***

In [2]:

params = {
    "host": "host.docker.internal",
    "dbname": "airflow",
    "port": 5532,
    "user": "postgres",
    "password": "postgres"
}
connection = None

try:
    connection = psycopg2.connect(**params)
    print("Connected to the database successfully.")
except Exception as e:
    print(f"An error occurred while connecting to the database: {e}")

if connection:

    btcHistDf = pd.read_sql_query("SELECT * FROM Historical_Market_Price", connection)
    print(btcHistDf.head())

    connection.close()
else:
    print("Unable to create a DataFrame as the connection was not established.")


An error occurred while connecting to the database: connection to server at "host.docker.internal" (10.1.10.121), port 5532 failed: Connection timed out (0x0000274C/10060)
	Is the server running on that host and accepting TCP/IP connections?

Unable to create a DataFrame as the connection was not established.


***Access Real_Time Data***

In [6]:
import psycopg2
import pandas as pd

params = {
    "host": "host.docker.internal",
    "dbname": "airflow",
    "port": 5532,
    "user": "postgres",
    "password": "postgres"
}
connection = None

try:
    connection = psycopg2.connect(**params)
    print("Connected to the database successfully.")
except Exception as e:
    print(f"An error occurred while connecting to the database: {e}")

if connection:

    btcRealDf = pd.read_sql_query("SELECT * FROM Real_Time_Market_Price", connection)
    # file_path = "real_time_market_price.csv"
    # df.to_csv(file_path, index=False)
    print(btcRealDf)

    connection.close()
else:
    print("Unable to create a DataFrame as the connection was not established.")


Connected to the database successfully.
    id   timestamp    values
0    1  1681430400  30407.60
1    2  1681430400  30407.60
2    3  1681344000  29899.24
3    4  1681430400  30407.60
4    5  1681430400  30407.60
5    6  1681430400  30407.60
6    7  1681430400  30407.60
7    8  1681430400  30407.60
8    9  1681430400  30407.60
9   10  1681430400  30407.60
10  11  1681430400  30407.60
11  12  1681430400  30407.60
12  13  1681430400  30407.60
13  14  1681430400  30407.60
14  15  1681430400  30407.60
15  16  1681430400  30407.60
16  17  1681430400  30407.60
17  18  1681430400  30407.60
18  19  1681430400  30407.60
19  20  1681430400  30407.60
20  21  1681430400  30407.60
21  22  1681430400  30407.60
22  23  1681430400  30407.60
23  24  1681430400  30407.60
24  25  1681430400  30407.60
25  26  1681430400  30407.60
26  27  1681430400  30407.60
27  28  1681430400  30407.60
28  29  1681430400  30407.60
29  30  1681430400  30407.60


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import plotly.graph_objects as go
from sklearn.ensemble import IsolationForest
from datetime import datetime, timedelta,timezone

# Data preprocessing

In [ ]:
btcHistDf['Timestamp'] = pd.to_datetime(btcHistDf.Timestamp, unit='s')
btcHistDf.head()

In [ ]:
btcRealDf['timestamp'] = pd.to_datetime(btcRealDf.timestamp, unit='s')
btcRealDf.head()

In [ ]:
print('Days in Historical dataset: ',len(btcHistDf))

In [ ]:
print('Days in Real-Time dataset: ',len(btcRealDf))

In [ ]:
histPano = btcHistDf.copy()
histPano = histPano.resample('D', on='Timestamp')[['Values']].mean()
fig = go.Figure()
fig.add_trace(go.Scatter(x=histPano.index, y=histPano['Values'],name='Full history BTC price'))
fig.update_layout(showlegend=True,title="BTC Historical Market Price",xaxis_title="Time",yaxis_title="Prices",font=dict(family="Courier New, monospace"))
fig.show()

In [ ]:
realPano = btcRealDf.copy()
realPano = realPano.resample('D', on='timestamp')[['values']].mean()
fig = go.Figure()
fig.add_trace(go.Scatter(x=realPano.index, y=realPano['values'],name='Full Real-Time BTC price'))
fig.update_layout(showlegend=True,title="BTC Real-Time Market Price",xaxis_title="Time",yaxis_title="Prices",font=dict(family="Courier New, monospace"))
fig.show()

# Data Anomalies

## Isolation forest model

# Preparing data to be passed to the Historical model
histOutliers = histPano.copy()
histOutliers.fillna(method ='bfill', inplace = True)

# Training the Historical model
hist_isolation_detector = IsolationForest(n_estimators=150,random_state=0,contamination='auto')
hist_isolation_detector.fit(histOutliers['Values'].values.reshape(-1, 1))

In [ ]:
# Preparing data to be passed to the Real-Time model
realOutliers = realPano.copy()
realOutliers.fillna(method ='bfill', inplace = True)

# Training the Real-Time model
real_isolation_detector = IsolationForest(n_estimators=150,random_state=0,contamination='auto')
real_isolation_detector.fit(realOutliers['values'].values.reshape(-1, 1))

In [ ]:
hist_data_ready = np.linspace(histOutliers['Values'].min(), histOutliers['Values'].max(), len(histOutliers)).reshape(-1,1)
histOutlier = hist_isolation_detector.predict(hist_data_ready)

In [ ]:
real_data_ready = np.linspace(realOutliers['values'].min(), realOutliers['values'].max(), len(realOutliers)).reshape(-1,1)
realOutlier = real_isolation_detector.predict(real_data_ready)

In [ ]:
histOutliers['outlier'] = histOutlier
histOutliers.head()

In [ ]:
realOutliers['outlier'] = realOutlier
realOutliers.head()

## Plotting prices' anomalies

In [ ]:
a = histOutliers.loc[histOutliers['outlier'] == 1] #anomaly
fig = go.Figure()
fig.add_trace(go.Scatter(x=histOutliers['Values'].index, y=histOutliers['Values'].values,mode='lines',name='BTC Price'))
fig.add_trace(go.Scatter(x=a.index, y=a['Values'].values,mode='markers',name='Anomaly',marker_symbol='x',marker_size=2))
fig.update_layout(showlegend=True,title="BTC Historical Market Price anomalies - IsolationForest",xaxis_title="Time",yaxis_title="Prices",font=dict(family="Courier New, monospace"))
fig.show()

In [ ]:
a = realOutliers.loc[realOutliers['outlier'] == 1] #anomaly
fig = go.Figure()
fig.add_trace(go.Scatter(x=realOutliers['values'].index, y=realOutliers['values'].values,mode='lines',name='BTC Price'))
fig.add_trace(go.Scatter(x=a.index, y=a['values'].values,mode='markers',name='Anomaly',marker_symbol='x',marker_size=2))
fig.update_layout(showlegend=True,title="BTC Real-Time Market Price anomalies - IsolationForest",xaxis_title="Time",yaxis_title="Prices",font=dict(family="Courier New, monospace"))
fig.show()

# Data Analysis

In [ ]:
histDataCheck = btcHistDf.groupby([btcHistDf['Timestamp'].dt.year])['Values'].mean()
histDataCheck

In [ ]:
realDataCheck = btcRealDf.groupby([btcRealDf['timestamp'].dt.year])['values'].mean()
realDataCheck

In [ ]:
f = plt.figure(figsize=(15,5))
ax = plt.gca()
histDataCheck.plot(ax=ax, kind='bar', figsize=(15,5))

In [ ]:
f = plt.figure(figsize=(15,5))
ax = plt.gca()
realDataCheck.plot(ax=ax, kind='bar', figsize=(15,5))